Copyright 2024 Google, LLC. This software is provided as-is,
without warranty or representation for any use or purpose. Your
use of it is subject to your agreement with Google.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# How to use Google Search Groundingwith Gemini

This notebook outlines how to interact with Vertex AI's Gemini models to ground results based on Google Search. More info can be found at https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini#web-ground-gemini

## Prepare the python development environment

First, let's identify any project specific variables to customize this notebook to your GCP environment. Change YOUR_PROJECT_ID with your own GCP project ID.

In [12]:
project_id = "YOUR_PROJECT_ID"
location = "global"
region = "us-central1"

Install any needed python modules from our requirements.txt file. Most Vertex Workbench environments include all the packages we'll be using, but if you are using an external Jupyter Notebook or require any additional packages for your own needs, you can simply add them to the included requirements.txt file an run the folloiwng commands.

In [13]:
#pip install -r requirements.txt

Now we will import all required modules. For our purpose, we will be utilizing the following:

- requests - This module will allow us to interact directly with external REST API's. 
- FunctionDeclaration - Used to define the function to be called by the model

In [14]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig, Part, Tool, ChatSession, FunctionDeclaration
import vertexai.preview.generative_models as generative_models
from vertexai.preview.generative_models import grounding
import requests

## Test the response without grounding

We will start by configuring our search variables and define the grounding tool. 

Define a tool to attach the functions to Gemini

In [15]:
search_tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

Initialize the model, first by NOT specifying the search grouding tool and simply using the foundational model.

In [16]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
)
chat = model.start_chat()

## Submit a prompt, call the function and return the response

In this example, we're using the external worldtimeapi.org api to find the current time in a specific timezone based on a specified area. The supported areas can be listed using running the following command 

Define a request to get the current time in Chicago

In [17]:
prompt = "Provide a detailed description of S&S item number 330-0016 "

Submit the prompt and print the response

In [18]:
response = chat.send_message(prompt)

Print the response and verify the model is not able to provide much detail without any grounding.

In [19]:
print(response.text)

Please provide me with more context about the item number 330-0016. I need to know what company or product line it belongs to in order to give you a detailed description. 

For example, is it:

* A part number for a specific manufacturer?
* An item number from a catalog?
* A product code from an online store?

Once you tell me more about where this item number comes from, I can help you find the information you need. 



## Test the response with grounding

Let's update the model to use Google Search as a grounding source

In [20]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[search_tool],
)
chat = model.start_chat()

In [21]:
response = chat.send_message(prompt)

Print the response using Google Search grounding and verify the information provided

In [22]:
print(response.text)

The S&S Cycle 509 Chain Drive Cam Kit (part numbers 330-0016, 141349) is designed for 1999-2006 Harley Davidson Big Twin Cam 88 engines with stock compression ratios. This kit provides increased torque in the low and mid-range RPM (1,000-4,500 RPM), with a lift of .509 inches. The kit includes two chain drive cams, bearings, o-rings, retaining ring, bolt, cam cover gasket, Loctite, and instructions. This product cannot be shipped to California. 

